In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_tasks, load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment, load_scheduling_queue
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

from omegaconf import OmegaConf
from pathlib import Path
from collections import defaultdict

In [ ]:
block = Budget.from_epsilon_delta(epsilon=10, delta=1e-8)

In [ ]:
from privacypacking.utils.zoo import zoo_df, build_zoo

In [ ]:
names_and_curves = build_zoo()
alphas_df, tasks_df = zoo_df(names_and_curves)

In [ ]:
alphas_df.head()

In [ ]:
indx = alphas_df.groupby('task_id')['normalized_epsilons'].idxmin()
best_alpha = alphas_df.loc[indx][["task_id","alphas"]]
best_alpha = best_alpha.rename(columns={"alphas":"best_alpha"})
best_alpha

In [ ]:
alphas_df = alphas_df.merge(best_alpha, how="outer", on="task_id")

In [ ]:
alphas_df.best_alphas.describe()

In [ ]:
px.line(
    alphas_df,
    x="alphas",
    y="normalized_epsilons",
    color="task_name",
    log_y=True,
    log_x=True,
)

In [ ]:
indx = alphas_df.groupby('task_id')['normalized_epsilons'].idxmin()
best_alpha = alphas_df.loc[indx]
px.scatter(
    best_alpha,
    x="alphas",
    y="normalized_epsilons",
    color="task_id",
    log_y=True,
    log_x=True,
    title="Epsilon for the best alpha of each task",
)

Let's see what curves are allocated by different schedulers

In [ ]:
df = load_latest_ray_experiment()

In [ ]:
# df = load_latest_scheduling_results(expname="ray/run_and_report_2022-04-05_22-11-57")
# TODO: custom load from Zoo

In [ ]:
df.columns